# Fabric Analytics Roadshow - Deployment Notebook

This notebook deploys all lab assets into your Fabric workspace.

## Prerequisites
- Access to a Microsoft Fabric workspace
- Contributor or Admin permissions in the workspace

## What This Notebook Does
1. Clones Analytics Roadshow GIT Repo to a temp directory
1. Install Pilot version of fabric-cicd library that supports Spark Job Definitions
1. Uses fabric-cicd python library to deploy the project to the target workspace

In [1]:
import tempfile
import subprocess
import os

# Create a temporary directory
temp_dir = tempfile.mkdtemp(prefix="fabric_roadshow_")
print(f"Created temporary directory: {temp_dir}")

# Clone the specific branch
repo_url = "https://github.com/microsoft/fabric-analytics-roadshow-lab.git"
branch_name = "initial-version-prep"

print(f"Cloning {repo_url} (branch: {branch_name})...")
result = subprocess.run(
    ["git", "clone", "--branch", branch_name, "--single-branch", repo_url, temp_dir],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print(f"✅ Successfully cloned repository to: {temp_dir}")
    workspace_root = os.path.join(temp_dir, "workspace")
    print(f"📁 Workspace directory: {workspace_root}")
else:
    print(f"❌ Error cloning repository:")
    print(result.stderr)
    raise Exception("Failed to clone repository")

Created temporary directory: C:\Users\MILESC~1\AppData\Local\Temp\fabric_roadshow_7uhjvfmj
Cloning https://github.com/microsoft/fabric-analytics-roadshow-lab.git (branch: initial-version-prep)...
✅ Successfully cloned repository to: C:\Users\MILESC~1\AppData\Local\Temp\fabric_roadshow_7uhjvfmj
📁 Workspace directory: C:\Users\MILESC~1\AppData\Local\Temp\fabric_roadshow_7uhjvfmj\workspace


In [3]:
# Install fabric_cicd from the cloned repository
whl_path = os.path.join(temp_dir, "setup", "fabric_cicd-0.1.33-py3-none-any.whl")
print(f"Installing fabric_cicd from: {whl_path}")

import subprocess
import sys

result = subprocess.run(
    [sys.executable, "-m", "pip", "install", whl_path, "--force-reinstall"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Successfully installed fabric_cicd")
else:
    print(f"❌ Installation failed:")
    print(result.stderr)

NameError: name 'temp_dir' is not defined

In [5]:
%pip install C:\Users\milescole\source\fabric-analytics-roadshow-lab\setup\fabric_cicd-0.1.33-py3-none-any.whl --force-reinstall

Defaulting to user installation because normal site-packages is not writeable
Processing c:\users\milescole\source\fabric-analytics-roadshow-lab\setup\fabric_cicd-0.1.33-py3-none-any.whl
  Using cached azure_identity-1.25.1-py3-none-any.whl.metadata (88 kB)
  Using cached dpath-2.2.0-py3-none-any.whl.metadata (15 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached jsonpath_ng-1.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached azure_core-1.37.0-py3-none-any.whl.metadata (47 kB)
  Using cached cryptography-46.0.3-cp311-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached msal-1.34.0-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadat

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


In [ ]:
from fabric_cicd import FabricWorkspace, publish_all_items, unpublish_all_orphan_items, append_feature_flag

from fabric_cicd import change_log_level
#change_log_level('DEBUG')
append_feature_flag("enable_lakehouse_unpublish")

# Sample values for FabricWorkspace parameters
workspace_id = "1e5cfd3c-ef5a-4cc1-8e22-2b3ed092c779"
# Use the workspace directory from the cloned repository
repository_directory = workspace_root

item_type_in_scope = ["Lakehouse", "Warehouse", "Notebook", "SparkJobDefinition", "Lakehouse", "Environment"]  # "" , "SparkJobDefinition", "Lakehouse", "Environment", 

# Initialize the FabricWorkspace object with the required parameters
target_workspace = FabricWorkspace(
    workspace_id=workspace_id,
    repository_directory=repository_directory,
    item_type_in_scope=item_type_in_scope
)

# Publish all items defined in item_type_in_scope
publish_all_items(target_workspace)

[info]   21:28:47 - Executing as User 'milescole@microsoft.com'
[debug]  21:28:47 - Using default parameter file path: 'C:\Users\milescole\source\fabric-analytics-roadshow-lab\workspace\parameter.yml'
[debug]  21:28:47 - Validation passed: YAML content is valid
[debug]  21:28:47 - Validating parameter file load
[debug]  21:28:47 - Validation passed: Successfully loaded parameter.yml
[debug]  21:28:47 - Validating parameter names
[debug]  21:28:47 - Validation passed: Parameter names are valid
[debug]  21:28:47 - Validating parameter file structure
[debug]  21:28:47 - Validation passed: Parameter file structure is valid
[debug]  21:28:47 - Validating find_replace parameter
[debug]  21:28:47 - Found the find_replace parameter
[debug]  21:28:47 - 8 find_replace parameters found
[debug]  21:28:47 - Validating find_replace 1 keys
[debug]  21:28:47 - Validation passed: find_replace contains valid keys
[debug]  21:28:47 - Validating find_replace 1 required values
[debug]  21:28:47 - Validatio


####################################################################################################
########## Validating Parameter File ###############################################################
####################################################################################################



[debug]  21:28:48 - 
URL: https://api.powerbi.com/v1/workspaces/1e5cfd3c-ef5a-4cc1-8e22-2b3ed092c779
Method: GET
Request Body:
"{}"
Response Status: 200
Response Headers:
{
    "Cache-Control": "no-store, must-revalidate, no-cache",
    "Pragma": "no-cache",
    "Content-Length": "286",
    "Content-Type": "application/json; charset=utf-8",
    "Content-Encoding": "gzip",
    "Strict-Transport-Security": "max-age=31536000; includeSubDomains",
    "X-Frame-Options": "deny",
    "X-Content-Type-Options": "nosniff",
    "RequestId": "a8baf8ea-9549-4cc0-a5e7-b512de88f527",
    "Access-Control-Expose-Headers": "RequestId",
    "request-redirected": "true",
    "home-cluster-uri": "https://df-msit-scus-redirect.analysis.windows.net/",
    "Date": "Tue, 06 Jan 2026 04:28:46 GMT"
}
Response Body:
{"id":"1e5cfd3c-ef5a-4cc1-8e22-2b3ed092c779","displayName":"AnalyticsRoadshow_TestDeploy6","description":"","type":"Workspace","capacityId":"43efd56c-2726-480a-b145-5fc98cf5c94e","capacityRegion":"Wes


####################################################################################################
########## Publishing Workspace Folders ############################################################
####################################################################################################



[debug]  21:28:49 - 
URL: https://api.powerbi.com/v1/workspaces/1e5cfd3c-ef5a-4cc1-8e22-2b3ed092c779/items
Method: GET
Request Body:
"{}"
Response Status: 200
Response Headers:
{
    "Cache-Control": "no-store, must-revalidate, no-cache",
    "Pragma": "no-cache",
    "Content-Length": "425",
    "Content-Type": "application/json; charset=utf-8",
    "Content-Encoding": "gzip",
    "Strict-Transport-Security": "max-age=31536000; includeSubDomains",
    "X-Frame-Options": "deny",
    "X-Content-Type-Options": "nosniff",
    "RequestId": "1d210d36-ee64-4642-90d5-2b1800ddd532",
    "Access-Control-Expose-Headers": "RequestId",
    "request-redirected": "true",
    "home-cluster-uri": "https://df-msit-scus-redirect.analysis.windows.net/",
    "Date": "Tue, 06 Jan 2026 04:28:49 GMT"
}
Response Body:
{"value":[{"id":"1f86418c-c49d-4af3-a5ee-28898854d05e","type":"SQLEndpoint","displayName":"SalesAndLogisticsLH","description":"","workspaceId":"1e5cfd3c-ef5a-4cc1-8e22-2b3ed092c779","sensitivity


####################################################################################################
########## Publishing Warehouses ###################################################################
####################################################################################################



[debug]  21:28:53 - 
URL: https://api.powerbi.com/v1/workspaces/1e5cfd3c-ef5a-4cc1-8e22-2b3ed092c779/items
Method: POST
Request Body:
{
    "displayName": "SalesAndLogisticsWH",
    "type": "Warehouse",
    "description": "",
    "folderId": ""
}
Response Status: 202
Response Headers:
{
    "Cache-Control": "no-store, must-revalidate, no-cache",
    "Pragma": "no-cache",
    "Content-Length": "24",
    "Content-Type": "application/json; charset=utf-8",
    "Content-Encoding": "gzip",
    "ETag": "\"\"",
    "Location": "https://df-msit-scus-redirect.analysis.windows.net/v1/operations/81e8c2d3-c011-4491-9523-a57dc504c859",
    "Retry-After": "20",
    "x-ms-operation-id": "81e8c2d3-c011-4491-9523-a57dc504c859",
    "Strict-Transport-Security": "max-age=31536000; includeSubDomains",
    "X-Frame-Options": "deny",
    "X-Content-Type-Options": "nosniff",
    "RequestId": "21ffbee5-9191-44b1-9390-08117f7bf5a9",
    "Access-Control-Expose-Headers": "RequestId,Location,Retry-After,ETag,x-ms-


####################################################################################################
########## Publishing Notebooks ####################################################################
####################################################################################################



[debug]  21:29:02 - 
URL: https://api.powerbi.com/v1/workspaces/1e5cfd3c-ef5a-4cc1-8e22-2b3ed092c779/items
Method: GET
Request Body:
"{}"
Response Status: 200
Response Headers:
{
    "Cache-Control": "no-store, must-revalidate, no-cache",
    "Pragma": "no-cache",
    "Content-Length": "463",
    "Content-Type": "application/json; charset=utf-8",
    "Content-Encoding": "gzip",
    "Strict-Transport-Security": "max-age=31536000; includeSubDomains",
    "X-Frame-Options": "deny",
    "X-Content-Type-Options": "nosniff",
    "RequestId": "976536d5-cf4a-4679-a1e0-8f0ef561e912",
    "Access-Control-Expose-Headers": "RequestId",
    "request-redirected": "true",
    "home-cluster-uri": "https://df-msit-scus-redirect.analysis.windows.net/",
    "Date": "Tue, 06 Jan 2026 04:29:01 GMT"
}
Response Body:
{"value":[{"id":"1f86418c-c49d-4af3-a5ee-28898854d05e","type":"SQLEndpoint","displayName":"SalesAndLogisticsLH","description":"","workspaceId":"1e5cfd3c-ef5a-4cc1-8e22-2b3ed092c779","sensitivity

In [9]:
# ANSI color codes
BLUE = "\033[34m"
CYAN = "\033[36m"
YELLOW = "\033[33m"
RED = "\033[31m"
BRIGHT_RED = "\033[1m\033[31m"
RESET = "\033[0m"

print(f"[debug]{RESET}   This is DEBUG - BLUE")
print(f"{CYAN}[info]{RESET}    This is INFO - CYAN")
print(f"{YELLOW}[warn]{RESET}    This is WARNING - YELLOW")
print(f"{RED}[error]{RESET}   This is ERROR - RED")
print(f"{BRIGHT_RED}[crit]{RESET}    This is CRITICAL - BRIGHT RED")

[debug]   This is DEBUG - BLUE
[info]    This is INFO - CYAN
[warn]    This is WARNING - YELLOW
[error]   This is ERROR - RED
[crit]    This is CRITICAL - BRIGHT RED
